In [23]:
from tabulate import tabulate

f = open("input.txt", "r")

workflows = {}

for line in f:
    if line =='\n':
        break
    line = line.replace('\n', '')
    [name, workflow] = line.split('{')
    workflow = workflow.replace('}', '')
    rules = workflow.split(',')
    workflow = []
    for rule in rules:
        if ':' in rule:
            [condition, destination] = rule.split(':')
            if '<' in condition:
                [rating, threshold] = condition.split('<')
                operator = '<'
            elif '>' in condition:
                [rating, threshold] = condition.split('>')
                operator = '>'
            else:
                raise ValueError
            threshold = int(threshold)
            workflow.append({"operator": operator, "rating": rating, "threshold": threshold, "destination": destination})
        else:
            destination = rule
            workflow.append({"destination": destination})
    workflows[name] = workflow

print(tabulate(workflows, headers="keys", tablefmt="rounded_grid"))

parts = []

for line in f:
    line = line.replace('\n', '').replace('{', '').replace('}', '')
    [x, m, a, s] = [int(rating.split('=')[1]) for rating in line.split(',')]
    parts.append({"x": x, "m": m, "a": a, "s": s})

print(tabulate(parts, headers="keys", tablefmt="rounded_grid"))

total = 0

for part_nb, part in enumerate(parts):
    processing_part = True
    workflow_name = "in"
    workflow = workflows[workflow_name]
    while processing_part:
        print(f"Processing part number {part_nb}")
        print(f"Workflow {workflow_name}")
        for rule in workflow:
            destination = rule["destination"]
            if "rating" not in rule:
                condition = True
            else:
                operator = rule["operator"]
                rating = rule["rating"]
                threshold = rule["threshold"]
                rating = part[rating]
                match operator:
                    case '<':
                        condition = (rating < threshold)
                    case '>':
                        condition = (rating > threshold)
                    case _:
                        raise
            if condition:
                if destination == 'A':
                    total += sum(part.values())
                    processing_part = False
                    print(f"Part number {part_nb} accepted")
                    break
                elif destination == 'R':
                    processing_part = False
                    print(f"Part number {part_nb} rejected")
                    break
                else:
                    workflow_name = destination
                    workflow = workflows[workflow_name]
                    break

print(total)

╭─────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────┬──────────────────────────────

In [77]:
from tabulate import TableFormat, tabulate
from math import prod
from copy import deepcopy

f = open("input.txt", "r")

workflows = {}

for line in f:
    if line =='\n':
        break
    line = line.replace('\n', '')
    [name, workflow] = line.split('{')
    workflow = workflow.replace('}', '')
    rules = workflow.split(',')
    workflow = []
    for rule in rules:
        if ':' in rule:
            [condition, destination] = rule.split(':')
            if '<' in condition:
                [rating, threshold] = condition.split('<')
                operator = '<'
            elif '>' in condition:
                [rating, threshold] = condition.split('>')
                operator = '>'
            else:
                raise ValueError
            threshold = int(threshold)
            workflow.append({"operator": operator, "rating": rating, "threshold": threshold, "destination": destination})
        else:
            destination = rule
            workflow.append({"destination": destination})
    workflows[name] = workflow

print(tabulate(workflows, headers="keys", tablefmt="rounded_grid"))

possibilities = {}

for workflow_name in workflows:
    workflow = workflows[workflow_name]
    # possibilities[workflow_name] will contain the possible ranges for ratings for each destination
    # reachable through workflow workflow_name 
    possibilities[workflow_name] = {}
    # Default ranges, without constraints
    rating_ranges= {"x": [1, 4000], "m": [1, 4000], "a": [1, 4000], "s": [1, 4000]}
    for rule in workflow:
        destination = rule["destination"]
        # For some workflows, destinations are reachable in multiple ways
        if destination not in possibilities[workflow_name]:
            possibilities[workflow_name][destination] = []
        rating_ranges_copy = deepcopy(rating_ranges)
        if "rating" not in rule:
            # A rule without condition, the ranges remain as they are
            possibilities[workflow_name][destination].append(rating_ranges_copy)
        else:
            # A rule with a condition
            rating = rule["rating"]
            operator = rule["operator"]
            threshold = rule["threshold"]
            # The ranges for the rating are adapted accordingly
            match operator:
                case '<':
                    rating_ranges_copy[rating][1] = threshold-1
                    rating_ranges[rating][0] = threshold
                case '>':
                    rating_ranges_copy[rating][0] = threshold+1
                    rating_ranges[rating][1] = threshold
                case _:
                    raise
            # We add the current ranges to the possibilities for this destination from this workflow
            possibilities[workflow_name][destination].append(rating_ranges_copy)

def possibilitesForAcceptance(possibilities, workflow_name, current_rating_ranges_list):
    total = 0
    # We go through all destinations that are susceptible to lead us to an "A" eventually 
    for destination, rating_ranges_list in possibilities[workflow_name].items():
        new_rating_ranges_list = []
        # For each destination we have a list of possiblities of ranges
        for rating_ranges in rating_ranges_list:
            # We called this function with a list of possibilities of ranges already set 
            # (from previous workflows that lead to this one)
            for current_rating_ranges in current_rating_ranges_list:
                new_rating_ranges = {}
                to_add = True
                for key in rating_ranges:
                    # For each rating, we merge the ranges we have
                    [current_min, current_max] = current_rating_ranges[key]
                    [workflow_min, workflow_max] = rating_ranges[key]
                    new_rating_ranges[key] = [max(current_min, workflow_min), min(current_max, workflow_max)]
                    if new_rating_ranges[key][1] < new_rating_ranges[key][0]:
                        # If the resulting range isn't valid, there's no need to go on, this set of ranges 
                        # can be thrown away
                        to_add = False
                        break
                if to_add:
                    # If we only had valid ranges, we add this set to the possibilities
                    new_rating_ranges_list.append(new_rating_ranges)
        if destination == 'A':
            # We've reached acceptance, now we need to compute the number of possible combinations of ratings
            # that could have lead here
            total_nb_of_poss = 0
            # Simply multiply the width of all the ranges for all the lists for destination 'A' 
            # and add them up together
            for rating_ranges in new_rating_ranges_list:
                nb_of_poss = 1
                for key in rating_ranges:
                    nb_of_poss *= (rating_ranges[key][1] - rating_ranges[key][0] + 1)
                total_nb_of_poss += nb_of_poss
            total += total_nb_of_poss
        elif destination == 'R':
            # We've reached rejection, no way to reach acceptance after this so we move on
            continue
        else:
            # This leads to another workflow, with an updated ranges list
            total += possibilitesForAcceptance(possibilities, destination, new_rating_ranges_list)
    return total

initial_rating_ranges= [{"x": [1, 4000], "m": [1, 4000], "a": [1, 4000], "s": [1, 4000]}]
# The first workflow is "in", with the initial constraints on the ratings
print(possibilitesForAcceptance(possibilities, "in", initial_rating_ranges))


╭─────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────┬──────────────────────────────